In [1]:
# For this exercise you will import COVID-19 global daily cases and daily confirmed deaths from a GitHub repository, apply some transformations to the data and push it to a SQL Database.
# 1-	In the first step you should load data from GitHub repository into two different pandas dataframes and display the data in two different tables.
# a.	Daily cases:
# https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv
# b.	Daily deaths:
# https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv

import pandas as pd

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

df_cases = pd.read_csv(url, error_bad_lines=False)

url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

df_deaths = pd.read_csv(url, error_bad_lines=False)


# 2-	You will need to apply several transformations to Daily cases dataframe and Daily deaths dataframe:
# a.	Daily cases:
# i.	Remove columns “Lat”,”Long”,”Province/State”

col_to_rem = ['Lat','Long','Province/State']

df_cases.drop(col_to_rem, axis=1, inplace=True)

In [2]:
# ii.	Transpose data by “Country/Region”
df_cases_transposed = df_cases.set_index(['Country/Region']).unstack().reset_index()
df_cases_transposed

,level_0,Country/Region,0
0,1/22/20,Afghanistan,0
1,1/22/20,Albania,0
2,1/22/20,Algeria,0
3,1/22/20,Andorra,0
4,1/22/20,Angola,0
...,...,...,...
89154,12/15/20,Vietnam,1405
89155,12/15/20,West Bank and Gaza,113409
89156,12/15/20,Yemen,2085
89157,12/15/20,Zambia,18428


In [3]:
# iii.	Rename columns:   
# 1.	“Country/Region”  “EntityName”
# 2.	Dates column ”Period”
# 3.	“value””Value”    
    
ren_col = {"Country/Region":"EntityName", "level_0":"Period", 0: "Value"}
    
df_cases_transposed= df_cases_transposed.rename(columns=ren_col)    
df_cases_transposed

,Period,EntityName,Value
0,1/22/20,Afghanistan,0
1,1/22/20,Albania,0
2,1/22/20,Algeria,0
3,1/22/20,Andorra,0
4,1/22/20,Angola,0
...,...,...,...
89154,12/15/20,Vietnam,1405
89155,12/15/20,West Bank and Gaza,113409
89156,12/15/20,Yemen,2085
89157,12/15/20,Zambia,18428


In [4]:
# iv.	Add a column called “Indicator” and fill it with the indicator name, in this case “COVID-19 confirmed cases’

df_cases_transposed["Indicator"] = "COVID-19 confirmed cases" 


In [5]:
# v.	Some country data in the original file was disaggregated by province. You will need to group by “EntityName, “Period” and “Indicator” and get the sum of each province to be the actual value for every period.
df_cases_grouped=df_cases_transposed.groupby(["EntityName", "Period", "Indicator"]).sum().reset_index()

df_cases_grouped


,EntityName,Period,Indicator,Value
0,Afghanistan,1/22/20,COVID-19 confirmed cases,0
1,Afghanistan,1/23/20,COVID-19 confirmed cases,0
2,Afghanistan,1/24/20,COVID-19 confirmed cases,0
3,Afghanistan,1/25/20,COVID-19 confirmed cases,0
4,Afghanistan,1/26/20,COVID-19 confirmed cases,0
...,...,...,...,...
62834,Zimbabwe,9/5/20,COVID-19 confirmed cases,6837
62835,Zimbabwe,9/6/20,COVID-19 confirmed cases,6837
62836,Zimbabwe,9/7/20,COVID-19 confirmed cases,7298
62837,Zimbabwe,9/8/20,COVID-19 confirmed cases,7388


In [6]:

# vi.	Make sure that datatypes for each column are:
# 1.	“EntityName” str
# 2.	“Indicator”str
# 3.	“Period” datetime64
# 4.	“Value” int64


print(df_cases_grouped.info())

df_cases_grouped['EntityName'] = df_cases_grouped['EntityName'].astype(str)
df_cases_grouped['Period'] = pd.to_datetime(df_cases_grouped['Period'])
df_cases_grouped['Indicator'] = df_cases_grouped['Indicator'].astype(str)
print(df_cases_grouped.dtypes)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62839 entries, 0 to 62838
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   EntityName  62839 non-null  object
 1   Period      62839 non-null  object
 2   Indicator   62839 non-null  object
 3   Value       62839 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.9+ MB
None
EntityName            object
Period        datetime64[ns]
Indicator             object
Value                  int64
dtype: object


In [7]:
# vii.	EntityNames in this dataframe must match names in the database.
names={"South Korea":"Korea",
"Bosnia and Herzegovina":"Bosnia-Herz.", "Congo (Kinshasa)":"DR Congo", "Cote d’Ivoire":"CdI",
"Czechia":"Czech Republic","Gambia":"Gambia,The","United Arab Emirates":"UAE","Taiwan*":"Taiwan","US":"United States"}

for i in range(len(df_cases_grouped["EntityName"])):
    for database, dataframe in names.items():        
        if df_cases_grouped["EntityName"][i] == database:
            df_cases_grouped["EntityName"][i] = dataframe
    
df_cases_grouped["EntityName"].unique()

C:\Users\crise\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia-Herz.', 'Botswana', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros',
       'Congo (Brazzaville)', 'DR Congo', 'Costa Rica', "Cote d'Ivoire",
       'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia,The', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guinea', 'G

In [8]:
# viii.	Export the dataframe as a .csv
df_cases_grouped.to_csv("Daily_cases.csv")


In [9]:
# b.	Daily deaths:
# i.	Repeat same steps you did for daily cases, but this time the indicator should be named “COVID-19 confirmed deaths”.


def convert_to_csv_and_clean(df, name_indicator, exit_csv):
    col_to_rem = ['Lat','Long','Province/State']
    df_nocol= df.drop(col_to_rem, axis=1)
    df_cases_transposed = df_nocol.set_index(['Country/Region']).unstack().reset_index()
    ren_col = {"Country/Region":"EntityName", "level_0":"Period", 0: "Value"}
    df_cases_transposed= df_cases_transposed.rename(columns=ren_col)  
    df_cases_transposed["Indicator"] = name_indicator 
    df_cases_grouped=df_cases_transposed.groupby(["EntityName", "Period", "Indicator"]).sum().reset_index()

    df_cases_grouped


    print(df_cases_grouped.info())
    
    df_cases_grouped['EntityName'] = df_cases_grouped['EntityName'].astype(str)
    df_cases_grouped['Period'] = pd.to_datetime(df_cases_grouped['Period'])
    df_cases_grouped['Indicator'] = df_cases_grouped['Indicator'].astype(str)
    print(df_cases_grouped.dtypes)

    names={"South Korea":"Korea",
    "Bosnia and Herzegovina":"Bosnia-Herz.", "Congo (Kinshasa)":"DR Congo", "Cote d’Ivoire":"CdI",
    "Czechia":"Czech Republic","Gambia":"Gambia,The","United Arab Emirates":"UAE","Taiwan*":"Taiwan","US":"United States"}

    for i in range(len(df_cases_grouped["EntityName"])):
        for database, dataframe in names.items():        
            if df_cases_grouped["EntityName"][i] == database:
                df_cases_grouped["EntityName"][i] = dataframe
    
    df_cases_grouped["EntityName"].unique()

    df_cases_grouped.to_csv(exit_csv)

In [10]:
#Mediante UDF
indicator_deaths="COVID-19 confirmed deaths"
csv_deaths="Confirmed_deaths.csv"
convert_to_csv_and_clean(df_deaths, indicator_deaths,csv_deaths)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62839 entries, 0 to 62838
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   EntityName  62839 non-null  object
 1   Period      62839 non-null  object
 2   Indicator   62839 non-null  object
 3   Value       62839 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.9+ MB
None
EntityName            object
Period        datetime64[ns]
Indicator             object
Value                  int64
dtype: object


C:\Users\crise\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
# 3-	Append both tables.
combined_csv = pd.concat([pd.read_csv("Daily_cases.csv"), pd.read_csv("Confirmed_deaths.csv")])

In [ ]:
# 4-	Insert the data from into a new table in a mock on-premises SQL Server database (since it’s not an actual database you can make up whatever names you want for the server, database, password, etc.).

import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="yourusername",
  password="yourpassword"
)

mycursor = mydb.cursor()

sql = "INSERT INTO covid *"

